In [1]:
#%% library import
import numpy as np
import pandas as pd
import networkx as nx
import torch as tc
import torch
import pprint
import pickle
import time

from torch.autograd import Variable
from sklearn.utils import shuffle
from sklearn.preprocessing import Normalizer
from matplotlib import pyplot as plt
from matplotlib import image as img
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from functools import partial

In [2]:
#%% mini_dataset loading
with open("datasets/KIBA/trainset_cnn+desc.txt", "rb") as fp:
    trainset = pickle.load(fp)

with open("datasets/KIBA/validset_cnn+desc.txt", "rb") as fp:
    validset = pickle.load(fp)

In [3]:
#%% Make collate func.
def collate(samples):
    # The input `samples` is a list of pairs [(graph, label),(graph, label)].
    descs, images, labels = map(list, zip(*samples))
    return tc.tensor(descs, dtype=tc.float).cuda(), tc.tensor(images, dtype=tc.float).cuda(), tc.tensor(labels).cuda()

In [4]:
#%% making model
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()    # method 상속받고 __init__()은 여기서 하겠다.
        
        self.dslayers = nn.Sequential(     
                        nn.Linear(9577, 2048),
                        nn.BatchNorm1d(2048),
                        nn.ReLU(),
                        nn.Dropout(0.2),
                        nn.Linear(2048, 512),
                        nn.BatchNorm1d(512),
                        nn.ReLU(),
                        nn.Dropout(0.2),
                        nn.Linear(512, 128),
                        )
    
        self.imlayers = nn.Sequential(
                        nn.Conv2d(1, 4, kernel_size = 3),
                        nn.Conv2d(4, 8, kernel_size = 3),
                        nn.Conv2d(8, 16, kernel_size = 5),
                        nn.BatchNorm2d(num_features = 16),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size = 2, stride = 2),
                        nn.Conv2d(16, 32, kernel_size = 5, stride = 1),
                        nn.BatchNorm2d(num_features = 32),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size = 2, stride = 2),
                        nn.Conv2d(32, 64, kernel_size = 3, stride = 1),
                        nn.BatchNorm2d(num_features = 64),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size = 2, stride = 2),
                        nn.Conv2d(64, 128, kernel_size = 3, stride = 2)
                        )
        
        self.avgpool = nn.AvgPool2d(15, count_include_pad=False)
        self.regress = nn.Linear(256, 1)

    def forward(self, desc, image):
        ds_h = self.dslayers(desc)
        dim = 1
        for d in ds_h.size()[1:]:
            dim = dim * d
        ds_h = ds_h.view(-1, dim)
            
        im_h = self.imlayers(image)
        im_h = self.avgpool(im_h)
        dim = 1
        for d in im_h.size()[1:]: #16, 4, 4
            dim = dim * d
        im_h = im_h.view(-1, dim)
                    
        cat = tc.cat((ds_h, im_h), axis=1).cuda()
       
        return self.regress(cat).cuda()

In [5]:
#%% Set hyperparameter
hp_d = {}

# FIXME: 학습 관련 하이퍼파라미터
hp_d['batch_size'] = 64
hp_d['num_epochs'] = 700

hp_d['init_learning_rate'] = 10 ** -3.70183
hp_d['eps'] = 10 ** -8.39981
hp_d['weight_decay'] = 10 ** -3.59967

In [6]:
#%% learning and validation
tr_data_loader = DataLoader(trainset, batch_size=hp_d['batch_size'], shuffle=False, collate_fn=collate)
va_data_loader = DataLoader(validset, batch_size=hp_d['batch_size'], shuffle=False, collate_fn=collate)

model = Regressor().to(torch.device('cuda:0'))
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
loss_func = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=hp_d['init_learning_rate'], 
    weight_decay=hp_d['weight_decay'], eps=hp_d['eps'])

print('tr_var:', np.var(np.array([s[2] for s in trainset])))
print('va_var:', np.var(np.array([s[2] for s in validset])))
print('total params:', total_params)

tr_epoch_losses = []
va_epoch_losses = []

start = time.time()

for epoch in range(hp_d['num_epochs']):                             #!! epoch-loop
    # training session
    model.train()
    tr_epoch_loss = 0

    for iter, (desc, image, label) in enumerate(tr_data_loader):    #!! batch-loop
        prediction = model(desc, image).view(-1)
        loss = loss_func(prediction, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        tr_epoch_loss += loss.detach().item()

    tr_epoch_loss /= (iter + 1)
    print('Training epoch {}, loss {:.4f}'.format(epoch, tr_epoch_loss))
    tr_epoch_losses.append(tr_epoch_loss)

    # validation session
    model.eval()
    va_epoch_loss = 0

    for iter, (desc, image, label) in enumerate(va_data_loader):  # batch-loop
        prediction = model(desc, image).view(-1)
        loss = loss_func(prediction, label)
        
        va_epoch_loss += loss.detach().item()
        
    va_epoch_loss /= (iter + 1)
    print('Validation epoch {}, loss {:.4f}'.format(epoch, va_epoch_loss))
    va_epoch_losses.append(va_epoch_loss)

end = time.time()
print('time elapsed:', end-start)

tr_var: 0.7886389921020749
va_var: 0.8894951422093866
total params: 20844833
Training epoch 0, loss 11.0885
Validation epoch 0, loss 1.0143
Training epoch 1, loss 0.8911
Validation epoch 1, loss 0.9900
Training epoch 2, loss 0.7928
Validation epoch 2, loss 1.1511
Training epoch 3, loss 0.7458
Validation epoch 3, loss 1.8348
Training epoch 4, loss 0.7259
Validation epoch 4, loss 1.9179
Training epoch 5, loss 0.7049
Validation epoch 5, loss 1.7041
Training epoch 6, loss 0.6852
Validation epoch 6, loss 7.2967
Training epoch 7, loss 0.6631
Validation epoch 7, loss 8.1787
Training epoch 8, loss 0.6427
Validation epoch 8, loss 6.1846
Training epoch 9, loss 0.6369
Validation epoch 9, loss 0.9441
Training epoch 10, loss 0.6222
Validation epoch 10, loss 1.9286
Training epoch 11, loss 0.6134
Validation epoch 11, loss 1.3188
Training epoch 12, loss 0.5952
Validation epoch 12, loss 0.6227
Training epoch 13, loss 0.5811
Validation epoch 13, loss 3.6876
Training epoch 14, loss 0.5688
Validation epoc

Training epoch 125, loss 0.2366
Validation epoch 125, loss 679.7026
Training epoch 126, loss 0.2368
Validation epoch 126, loss 624.5264
Training epoch 127, loss 0.2334
Validation epoch 127, loss 802.2182
Training epoch 128, loss 0.2348
Validation epoch 128, loss 639.5465
Training epoch 129, loss 0.2408
Validation epoch 129, loss 1165.7621
Training epoch 130, loss 0.2433
Validation epoch 130, loss 1385.2956
Training epoch 131, loss 0.2414
Validation epoch 131, loss 1702.5846
Training epoch 132, loss 0.2410
Validation epoch 132, loss 833.2726
Training epoch 133, loss 0.2399
Validation epoch 133, loss 104.4660
Training epoch 134, loss 0.2375
Validation epoch 134, loss 6.9884
Training epoch 135, loss 0.2351
Validation epoch 135, loss 5.8786
Training epoch 136, loss 0.2334
Validation epoch 136, loss 4.8617
Training epoch 137, loss 0.2341
Validation epoch 137, loss 22.1207
Training epoch 138, loss 0.2334
Validation epoch 138, loss 40.7490
Training epoch 139, loss 0.2318
Validation epoch 139,

Training epoch 247, loss 0.2242
Validation epoch 247, loss 148.7171
Training epoch 248, loss 0.2256
Validation epoch 248, loss 28.2299
Training epoch 249, loss 0.2257
Validation epoch 249, loss 48.2076
Training epoch 250, loss 0.2234
Validation epoch 250, loss 126.0532
Training epoch 251, loss 0.2239
Validation epoch 251, loss 377.3161
Training epoch 252, loss 0.2260
Validation epoch 252, loss 513.0022
Training epoch 253, loss 0.2258
Validation epoch 253, loss 512.0824
Training epoch 254, loss 0.2264
Validation epoch 254, loss 673.2661
Training epoch 255, loss 0.2258
Validation epoch 255, loss 675.3287
Training epoch 256, loss 0.2261
Validation epoch 256, loss 640.3224
Training epoch 257, loss 0.2269
Validation epoch 257, loss 495.6792
Training epoch 258, loss 0.2278
Validation epoch 258, loss 501.7369
Training epoch 259, loss 0.2257
Validation epoch 259, loss 361.0786
Training epoch 260, loss 0.2249
Validation epoch 260, loss 24.4555
Training epoch 261, loss 0.2266
Validation epoch 26

Training epoch 369, loss 0.2213
Validation epoch 369, loss 42.4984
Training epoch 370, loss 0.2194
Validation epoch 370, loss 11.9991
Training epoch 371, loss 0.2192
Validation epoch 371, loss 88.3114
Training epoch 372, loss 0.2205
Validation epoch 372, loss 86.4350
Training epoch 373, loss 0.2218
Validation epoch 373, loss 163.9862
Training epoch 374, loss 0.2219
Validation epoch 374, loss 235.1121
Training epoch 375, loss 0.2217
Validation epoch 375, loss 299.5102
Training epoch 376, loss 0.2197
Validation epoch 376, loss 262.0749
Training epoch 377, loss 0.2194
Validation epoch 377, loss 376.3898
Training epoch 378, loss 0.2189
Validation epoch 378, loss 282.9162
Training epoch 379, loss 0.2195
Validation epoch 379, loss 2.2764
Training epoch 380, loss 0.2211
Validation epoch 380, loss 0.7493
Training epoch 381, loss 0.2200
Validation epoch 381, loss 2.8416
Training epoch 382, loss 0.2213
Validation epoch 382, loss 3.5627
Training epoch 383, loss 0.2199
Validation epoch 383, loss 1

Training epoch 491, loss 0.2131
Validation epoch 491, loss 94.3925
Training epoch 492, loss 0.2129
Validation epoch 492, loss 181.5480
Training epoch 493, loss 0.2118
Validation epoch 493, loss 204.3472
Training epoch 494, loss 0.2124
Validation epoch 494, loss 297.7880
Training epoch 495, loss 0.2131
Validation epoch 495, loss 51.8048
Training epoch 496, loss 0.2126
Validation epoch 496, loss 3.8779
Training epoch 497, loss 0.2151
Validation epoch 497, loss 67.5784
Training epoch 498, loss 0.2145
Validation epoch 498, loss 87.1550
Training epoch 499, loss 0.2154
Validation epoch 499, loss 65.8559
Training epoch 500, loss 0.2147
Validation epoch 500, loss 69.2523
Training epoch 501, loss 0.2152
Validation epoch 501, loss 96.9738
Training epoch 502, loss 0.2148
Validation epoch 502, loss 9.6437
Training epoch 503, loss 0.2150
Validation epoch 503, loss 13.6773
Training epoch 504, loss 0.2143
Validation epoch 504, loss 12.5241
Training epoch 505, loss 0.2137
Validation epoch 505, loss 28

Validation epoch 613, loss 0.6092
Training epoch 614, loss 0.2104
Validation epoch 614, loss 21.2943
Training epoch 615, loss 0.2105
Validation epoch 615, loss 124.5692
Training epoch 616, loss 0.2097
Validation epoch 616, loss 197.2831
Training epoch 617, loss 0.2089
Validation epoch 617, loss 202.1692
Training epoch 618, loss 0.2088
Validation epoch 618, loss 149.7241
Training epoch 619, loss 0.2095
Validation epoch 619, loss 56.9102
Training epoch 620, loss 0.2094
Validation epoch 620, loss 1.0656
Training epoch 621, loss 0.2085
Validation epoch 621, loss 0.5035
Training epoch 622, loss 0.2096
Validation epoch 622, loss 0.4723
Training epoch 623, loss 0.2080
Validation epoch 623, loss 27.8251
Training epoch 624, loss 0.2077
Validation epoch 624, loss 88.9041
Training epoch 625, loss 0.2076
Validation epoch 625, loss 116.0746
Training epoch 626, loss 0.2090
Validation epoch 626, loss 4.1076
Training epoch 627, loss 0.2084
Validation epoch 627, loss 25.7097
Training epoch 628, loss 0.

In [7]:
np.save('vanilacnn+desc_tr_losses', tr_epoch_losses)
np.save('vanilacnn+desc_va_losses', va_epoch_losses)

18752.212121180124